In [21]:
import http.client
import os
import sys
from time import sleep
from time import time as now
from notebook.auth import passwd

import requests
from bs4 import BeautifulSoup
from requests.packages.urllib3.exceptions import SubjectAltNameWarning

In [59]:
# Set this to 1 for debugging HTTP server communication
http.client.HTTPConnection.debuglevel = 0
# suppress SubjectAltNameWarning
requests.packages.urllib3.disable_warnings(SubjectAltNameWarning)

In [60]:
ec2_username = "automatedservices"
ec2_password = "EC2RO-ResMed123+"

In [61]:
bamboo_base_url = "https://bamboo.ec2.local"

security_certificate_path = os.path.expanduser("~/Americas-Root.pem")
# bamboo_build_url = "https://bamboo.ec2.local/artifact/MON-ATEST/shared/build-911/"
bamboo_build_url = 'https://bamboo.ec2.local/artifact/MON-ATEST/shared/build-2058/'
local_dir = 'traceability_local_data/'

In [62]:
# this function tries to make an https request 3 times before giving up, displaying an error message, and moving on
def make_https_request(url, security_certificate_path, attempt_number=0):
    if attempt_number > 2:
        print(f'TIMED OUT AFTER {attempt_number} ATTEMPTS TO CONNECT TO {url}!')
        return None
    try:
        return requests.get(url, auth=(ec2_username, ec2_password), verify=security_certificate_path)
    except requests.exceptions.ConnectionError:
        sleep(1)
        return make_https_request(url, security_certificate_path, attempt_number + 1)

In [63]:
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    return directory

In [64]:
def save_file(response, file_name):
    with open(file_name, 'w') as f:
        f.write(response.content)

In [65]:
def get_dir_name_from_path(file_name):
    """
    Example:

    :param file_name: bamboo.ec2.local/artifact/MON-ATEST/shared/build-2058/Acceptance-Test-Results-01
    :returns: 01
    """
    return file_name.split('-')[-1]


In [72]:
def download_reports():
    print("Downloading reports...", end='')

    response = make_https_request('https://bamboo.ec2.local/artifact/MON-ATEST/shared/build-2058/Acceptance-Test-Results-01/cucumberReports/report.json', security_certificate_path)
    print('done!')
    
    return response.content.decode()

In [79]:
report_string = download_reports()
type(report)

str

In [75]:
directory = create_dir(os.path.join(local_dir, get_dir_name_from_path('bamboo.ec2.local/artifact/MON-ATEST/shared/build-2058/Acceptance-Test-Results-01')))

In [76]:
directory

'traceability_local_data/01'

In [77]:
with open(os.path.join(directory, 'report.json'), mode='w') as f:
    json.dump(report_string, f)

In [ ]:
transform_url = 'https://taas-traceability-service.apps.us6.paas.easycareonline.com/taas/traceability/report/transform'
x = requests.post(transform_url, data = json.loads(report_string))

In [94]:
import requests

url = 'https://www.w3schools.com/python/demopage.php'
myjson = {'somekey': 'somevalue'}

x = requests.post(url, json = myjson)

#print the response text (the content of the requested file):

x.headers

{'Content-Encoding': 'gzip', 'Cache-Control': 'public', 'Content-Type': 'text/html', 'Date': 'Thu, 21 Nov 2019 03:41:45 GMT', 'Server': 'Microsoft-IIS/7.5', 'Vary': 'Accept-Encoding', 'X-Frame-Options': 'SAMEORIGIN', 'X-Powered-By': 'PHP/5.4.2, ASP.NET', 'Content-Length': '119'}

In [89]:
help(requests.Response)

Help on class Response in module requests.models:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |  
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if 